Mount google drive

Upload the tensorflow folder with correct directory to google drive

Similar to this reference link

https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

Remember to change your runtime to GPU for training.

In [ ]:
# Step 1 - Mount Gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install all the required dependencies

In [ ]:
# Step 2 - Install dependencies
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis
!pip install avro-python3==1.8.2

Protobuf Installation/Compilation

In [ ]:
#Step 3 - Protobuf Installation
#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/Tensorflow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/Tensorflow/models/research


Since using a VM, have to specific the path

In [ ]:
#Step 4- Set the environment.

import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/Tensorflow/models"
sys.path.append("/content/gdrive/My Drive/Tensorflow/models/research")

In [ ]:
#Step 5- Build and Install setup.py.

!python setup.py build
!python setup.py install

Navigate to dedicated project, then start the tensorboard for training evaluation

In [ ]:
#Step 6 - Tensorboard
%cd '/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

Change the pipeline.config parameters to yield optimum training result.

In [ ]:
# Step 7 - Training
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

In [ ]:
#Step 8- Test the Model.

#Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/exported-models/my_model/saved_model"

print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

Loading model...Done!


In [ ]:
#Step 9- Testing the Model.
from object_detection.utils import label_map_util

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/annotations/label_map.pbtxt",use_display_name=True)

#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

In [ ]:
#Step 10- Testing the Model.

#Loading the image
img=['/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/images/train/1.jpg','/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/images/train/500.jpg']
print(img)

#list containing paths of all the images

['/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/images/train/1.jpg', '/content/gdrive/My Drive/Tensorflow/workspace/Traffic_cone/images/train/500.jpg']


In [ ]:
#Step 11- Running the Inference.

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()

Once model is trained, it can be export to deploy on other machine

In [ ]:
# Step 12 - Export Model
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir=models/my_ssd_resnet50_v1_fpn/ --output_directory=exported-models/my_model